In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import ast
import numpy as np

In [ ]:
df = pd.read_csv('poses.csv')

In [ ]:
df['keypoints'] = df['keypoints'].apply(ast.literal_eval)
df['angle_vec'] = df['angle_vec'].apply(ast.literal_eval)
df['angle_score'] = df['angle_score'].apply(ast.literal_eval)
df['bbox'] = df['bbox'].apply(ast.literal_eval)

In [ ]:
from emv.features.pose import filter_poses

In [ ]:
# this will potentially run for a long time
df_filtered = filter_poses(df)

In [ ]:
df.shape, df_filtered.shape

In [ ]:
df_filtered.to_csv('poses_filtered.csv', index=False)

In [ ]:
df_filtered = pd.read_csv('poses_filtered.csv')

In [ ]:
df_filtered['angle_vec'] = df_filtered['angle_vec'].apply(ast.literal_eval)

In [ ]:
df_filtered.head()

In [ ]:
from emv.db.dao import DataAccessObject
from sqlalchemy.sql import text

In [ ]:
sports = []

for i, row in df_filtered.iterrows():
    query = text("""SELECT * FROM media WHERE media_id = :media_id""")
    res = DataAccessObject().fetch_all(query, {'media_id': row.media_id})
    sports.append(res[0]['metadata']['sport'])

In [ ]:
df_filtered['sport'] = sports

In [ ]:
data = np.array(df_filtered.angle_vec.values.tolist())

In [ ]:
import umap

reducer = umap.UMAP()
embedding = reducer.fit_transform(data)

In [ ]:
df_filtered

In [ ]:
df_filtered.sports.nunique()

In [ ]:
df_filtered.sports.nunique()

In [ ]:
# plot the embedding
import matplotlib.pyplot as plt

colors = df_filtered['sport'].apply(lambda x: ['red', 'blue', 'green', 'yellow', 'black'][x])

plt.scatter(embedding[:, 0], embedding[:, 1], c=df_filtered['sport'])